Cleaning May 5th 2022 data!

In [ ]:
import pandas as pd
import numpy as np
import ast
from math import radians, cos, sin, asin, sqrt

# Read the CSV file
df = pd.read_csv('../datasets/old/tripdata_2022_05_05.csv')

In [ ]:
# Drop unused columns
columns_to_drop = [
    'fecha', 'idBike', 'fleet', 'locktype', 'unlocktype',
    'address_lock', 'address_unlock', 'dock_unlock', 'dock_lock'
]
df = df.drop(columns=columns_to_drop)

In [ ]:
# Extract coordinates
def extract_coordinates(geo_str):
    try:
        geo_dict = ast.literal_eval(geo_str)
        lon, lat = geo_dict['coordinates']
        return pd.Series([lat, lon])
    except:
        return pd.Series([None, None])

df[['unlock_lat', 'unlock_lon']] = df['geolocation_unlock'].apply(extract_coordinates)
df[['lock_lat', 'lock_lon']] = df['geolocation_lock'].apply(extract_coordinates)

In [ ]:
# Create all_stations.csv
station_df = df[['station_unlock', 'unlock_station_name', 'unlock_lat', 'unlock_lon']].drop_duplicates()
station_df = station_df.rename(columns={
    'station_unlock': 'station_id',
    'unlock_station_name': 'station_name',
    'unlock_lat': 'lat',
    'unlock_lon': 'lon'
})

# Drop rows with missing values
station_df = station_df.dropna()

# Sort by station_id
station_df = station_df.sort_values(by='station_id')

station_df.to_csv('../datasets/all_stations.csv', index=False)

In [ ]:
# Create all_trips.csv
trip_df = df[[
    'id', 'unlock_date', 'lock_date',
    'station_unlock', 'station_lock',
    'trip_minutes', 'unlock_lat', 'unlock_lon',
    'lock_lat', 'lock_lon'
]]

trip_df = trip_df.rename(columns={
    'id': 'trip_id',
    'unlock_date': 'start_time',
    'lock_date': 'end_time',
    'station_unlock': 'start_station_id',
    'station_lock': 'end_station_id'
})

# Convert time columns to datetime
trip_df['start_time'] = pd.to_datetime(trip_df['start_time'])
trip_df['end_time'] = pd.to_datetime(trip_df['end_time'])

# Remove rows with missing values
trip_df = trip_df.dropna()

# Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Calculate distance for each trip
trip_df['distance_km'] = trip_df.apply(
    lambda row: haversine(row['unlock_lat'], row['unlock_lon'], row['lock_lat'], row['lock_lon']),
    axis=1
)

# Save trips
trip_df.to_csv('../datasets/all_trips_05_05.csv', index=False)

Cleaning May 11th 2022 data!

In [ ]:
import pandas as pd
import numpy as np
import ast
from math import radians, cos, sin, asin, sqrt

# Read the CSV file
df = pd.read_csv('../datasets/old/tripdata_2022_05_11.csv')

In [ ]:
# Drop unused columns
columns_to_drop = [
    'fecha', 'idBike', 'fleet', 'locktype', 'unlocktype',
    'address_lock', 'address_unlock', 'dock_unlock', 'dock_lock'
]
df = df.drop(columns=columns_to_drop)

In [ ]:
# Extract coordinates
def extract_coordinates(geo_str):
    try:
        geo_dict = ast.literal_eval(geo_str)
        lon, lat = geo_dict['coordinates']
        return pd.Series([lat, lon])
    except:
        return pd.Series([None, None])

df[['unlock_lat', 'unlock_lon']] = df['geolocation_unlock'].apply(extract_coordinates)
df[['lock_lat', 'lock_lon']] = df['geolocation_lock'].apply(extract_coordinates)

In [ ]:
# Create all_trips_05_11.csv
trip2_df = df[[
    'id', 'unlock_date', 'lock_date',
    'station_unlock', 'station_lock',
    'trip_minutes', 'unlock_lat', 'unlock_lon',
    'lock_lat', 'lock_lon'
]]

trip2_df = trip2_df.rename(columns={
    'id': 'trip_id',
    'unlock_date': 'start_time',
    'lock_date': 'end_time',
    'station_unlock': 'start_station_id',
    'station_lock': 'end_station_id'
})

# Convert time columns to datetime
trip2_df['start_time'] = pd.to_datetime(trip2_df['start_time'])
trip2_df['end_time'] = pd.to_datetime(trip2_df['end_time'])

# Remove rows with missing values
trip2_df = trip2_df.dropna()

# Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Calculate distance for each trip
trip2_df['distance_km'] = trip2_df.apply(
    lambda row: haversine(row['unlock_lat'], row['unlock_lon'], row['lock_lat'], row['lock_lon']),
    axis=1
)

# Save trips
trip2_df.to_csv('../datasets/all_trips_05_11.csv', index=False)

In [ ]:
# Removing all data that is not between 00:00:00 and 23:59:59 for both days
import pandas as pd

# Load the CSV files with datetime parsing
df_05_05 = pd.read_csv("../datasets/all_trips_05_05.csv", parse_dates=["start_time", "end_time"])
df_05_11 = pd.read_csv("../datasets/all_trips_05_11.csv", parse_dates=["start_time", "end_time"])

# Define the valid cutoff times
cutoff_05_05 = pd.Timestamp("2022-05-05 23:59:59")
cutoff_05_11 = pd.Timestamp("2022-05-11 23:59:59")

# Filter out trips ending after midnight
df_05_05 = df_05_05[df_05_05["end_time"] <= cutoff_05_05]
df_05_11 = df_05_11[df_05_11["end_time"] <= cutoff_05_11]

# Overwrite the original files with cleaned data
df_05_05.to_csv("../datasets/all_trips_05_05.csv", index=False)
df_05_11.to_csv("../datasets/all_trips_05_11.csv", index=False)

In [ ]:

# Converting the old stations ids (int) to the ones corresponding them from the station name (str)
import pandas as pd

stations_df = pd.read_csv("../datasets/all_stations.csv")

# This grabs everything before the first space or " -"
stations_df['new_station_id'] = stations_df['station_name'].str.extract(r'^(\S+)')

station_id_mapping = stations_df[['station_id', 'new_station_id']].copy()
station_id_mapping.columns = ['old_station_id', 'new_station_id']
station_id_mapping = station_id_mapping.dropna()

# Convert old ID to float (in case it’s stored as "69.0" in trips)
station_id_mapping['old_station_id'] = station_id_mapping['old_station_id'].astype(float)

# Ensure new ID is string (it will be anyway, but explicitly)
station_id_mapping['new_station_id'] = station_id_mapping['new_station_id'].astype(str)

station_id_mapping.to_csv("../datasets/old/station_id_mapping.csv", index=False)

In [ ]:
# Map the new stations ids to the old ones in all files
import pandas as pd

# Load mapping
mapping_df = pd.read_csv("../datasets/old/station_id_mapping.csv")
id_map = mapping_df.set_index('old_station_id')['new_station_id'].to_dict()

stations_df = pd.read_csv("../datasets/all_stations.csv")
trips_05_05 = pd.read_csv("../datasets/all_trips_05_05.csv")
trips_05_11 = pd.read_csv("../datasets/all_trips_05_11.csv")

# Update station_id in stations file
stations_df['station_id'] = stations_df['station_id'].astype(float).map(id_map)

# Update start and end station IDs in both trip files
for df in [trips_05_05, trips_05_11]:
    df['start_station_id'] = df['start_station_id'].astype(float).map(id_map)
    df['end_station_id'] = df['end_station_id'].astype(float).map(id_map)

    # Drop rows that failed to map
    df.dropna(subset=['start_station_id', 'end_station_id'], inplace=True)

stations_df.to_csv("../datasets/all_stations.csv", index=False)
trips_05_05.to_csv("../datasets/all_trips_05_05.csv", index=False)
trips_05_11.to_csv("../datasets/all_trips_05_11.csv", index=False)